In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.backends.mps.is_available()

True

In [5]:
tensor0d = torch.tensor(1.0)
tensor1d = torch.tensor([1.0, 2.0, 3.0])
tensor2d = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
tensor3d = torch.tensor([[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], [[7.0, 8.0, 9.0], [10.0, 11.0, 12.0]]])

print(tensor0d)
print(tensor1d)
print(tensor2d)
print(tensor3d)

tensor(1.)
tensor([1., 2., 3.])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])


In [7]:
tensor2d = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
print(tensor2d.T)

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])


In [8]:
print(tensor2d @ tensor2d.T)

tensor([[14., 32.],
        [32., 77.]])


In [9]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

In [13]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = torch.autograd.grad(loss, w1, retain_graph=True)
grad_L_b = torch.autograd.grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

loss.backward()
print(w1.grad)
print(b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor([-0.0898])
tensor([-0.0817])


In [16]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_imputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # Input layer
            torch.nn.Linear(num_imputs, 30),
            torch.nn.ReLU(),

            # Hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # Output layer
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork(num_imputs=50, num_outputs=3)
print(model)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params} trainable parameters.')

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
The model has 2213 trainable parameters.


In [18]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)

Parameter containing:
tensor([[-0.0937,  0.0906,  0.1248,  ..., -0.0616,  0.1058, -0.0411],
        [-0.0620,  0.0243, -0.0890,  ...,  0.0399,  0.0024,  0.1234],
        [ 0.1074,  0.0324,  0.1107,  ..., -0.1115,  0.1334,  0.0914],
        ...,
        [ 0.0317,  0.0826,  0.0567,  ..., -0.0938, -0.0183,  0.0777],
        [ 0.1405, -0.1073, -0.0538,  ..., -0.1038, -0.1146, -0.0132],
        [ 0.1167, -0.0787,  0.0347,  ...,  0.0791,  0.0636,  0.0732]],
       requires_grad=True)
torch.Size([30, 50])


In [19]:
torch.manual_seed(123)
x = torch.rand((1, 50))
out = model(x)
print(out)

tensor([[ 0.0184, -0.0739, -0.2149]], grad_fn=<AddmmBackward0>)


In [20]:
# 推論だけの場合
with torch.no_grad():
    out = model(x)
    print(out)

tensor([[ 0.0184, -0.0739, -0.2149]])
